In [15]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import time
import os

os.environ["http_proxy"] = "http://proxy.uec.ac.jp:8080/"
os.environ["https_proxy"] = "http://proxy.uec.ac.jp:8080/"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [16]:
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

batch_size = 128

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [17]:
import torch
from vit_pytorch import ViT
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = ViT(
    image_size=32,
    patch_size=4,
    num_classes=10,
    dim=256,
    depth=3,
    heads=4,
    mlp_dim=256,
    dropout=0.1,
    emb_dropout=0.1
).to(device)

net = torch.nn.DataParallel(net, device_ids=[0,1])

In [18]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
epochs = 20

for epoch in tqdm(range(0, epochs)):
    epoch_train_loss = 0
    epoch_train_acc = 0
    epoch_test_loss = 0
    epoch_test_acc = 0
    net.train()
    for data in train_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()/len(train_loader)
        acc = (outputs.argmax(dim=1) == labels).float().mean()
        epoch_train_acc += acc/len(train_loader)
    net.eval()
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            epoch_test_loss += loss.item()/len(test_loader)
            test_acc = (outputs.argmax(dim=1) == labels).float().mean()
            epoch_test_acc += test_acc/len(test_loader)
    print(f'Epoch {epoch+1} : train acc. {epoch_train_acc:.2f} train loss {epoch_train_loss:.2f}')
    print(f'Epoch {epoch+1} : test acc. {epoch_test_acc:.2f} test loss {epoch_test_loss:.2f}')

  5%|▌         | 1/20 [00:09<02:56,  9.31s/it]

Epoch 1 : train acc. 0.15 train loss 2.26
Epoch 1 : test acc. 0.19 test loss 2.17


 10%|█         | 2/20 [00:17<02:41,  8.95s/it]

Epoch 2 : train acc. 0.19 train loss 2.15
Epoch 2 : test acc. 0.23 test loss 2.09


 15%|█▌        | 3/20 [00:25<02:27,  8.68s/it]

...

Epoch 18 : train acc. 0.39 train loss 1.71
Epoch 18 : test acc. 0.41 test loss 1.66


 95%|█████████▌| 19/20 [02:34<00:08,  8.13s/it]

Epoch 19 : train acc. 0.39 train loss 1.70
Epoch 19 : test acc. 0.41 test loss 1.64


100%|██████████| 20/20 [02:42<00:00,  8.09s/it]

Epoch 20 : train acc. 0.40 train loss 1.68
Epoch 20 : test acc. 0.43 test loss 1.63


### 結果
- Epoch 20 : train acc. 0.40 train loss 1.68
- Epoch 20 : test acc. 0.43 test loss 1.63
- CNN等と比較しても，単純なViTのCIFAR-10への適用は，精度が低い．